On prend le problème de prédiction du débit moyen journalier à une semaine, le critère que l'on va minimiser est un mean square error sur toutes les stations. (discuter)

On choisis de faire de la validation croisée pour faire de la sélection de modèle, et on garde une ensemble de test.  
On a des données de 2010 à 2020 (11 années).  

On garde 2020 comme ensemble de test, et on fait 10 ensembles de validation croisée, et on utilise le split time series de sklearn (prend le début et tester sur l'année suivante), ca fait une erreur calculée sur 9 splits.

Pour les évaluations, la première prédiction est celle du 15 Janvier, que l'on prédit à partir des données du 1 au 7 Janvier, on se place le 8 à 00h00. (on laisse une semaine de données pour calculer les features, et on prédit à J+7).

In [1]:
import pandas as pd
import numpy as np

In [7]:
stations_debit = pd.read_csv("../Data/Base/Stations_Debit.csv")
liste_stations = list(stations_debit["Code station"])
mesures = pd.read_csv("../Data/Base/Mesures.csv")
mesures["Date"] = pd.to_datetime(mesures["Date"], format = "%Y/%m/%d %H:%M:%S")

## Ensemble de test

In [8]:
mesures_test = mesures[mesures["Date"].apply(lambda x: x.year == 2020)]

In [9]:
mesures_test_X = mesures_test.copy()
mesures_test_X.to_csv("../Data/Base/Mesures_Test_X.csv",
                     index = False)
mesures_test_X

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Vitesse_vent_07747,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Direction_vent_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
43824,2020-01-01 00:00:00,60.0,197.0,582.0,1110.0,1.47,6.30,19.4,46.2,88.2,...,2.366667,4.400000,89.666667,0.0,103053.333333,43.333333,2.800000,6.766667,85.666667,0.000000
43825,2020-01-01 02:00:00,56.8,193.0,570.0,1110.0,1.46,6.25,19.2,44.9,88.5,...,2.333333,3.566667,90.333333,0.0,103043.333333,40.000000,2.866667,6.133333,86.000000,0.000000
43826,2020-01-01 04:00:00,61.5,185.0,577.0,1100.0,1.46,6.21,19.0,41.1,88.2,...,1.600000,3.500000,91.000000,0.0,103030.000000,40.000000,3.400000,5.400000,88.000000,0.000000
43827,2020-01-01 06:00:00,68.6,186.0,600.0,1080.0,1.45,6.16,18.9,41.2,87.9,...,2.066667,2.500000,89.000000,0.0,103010.000000,33.333333,3.933333,4.600000,88.666667,0.000000
43828,2020-01-01 08:00:00,56.7,193.0,607.0,1060.0,1.44,6.13,18.8,44.2,87.9,...,2.133333,4.133333,84.000000,0.0,103043.333333,30.000000,3.633333,5.300000,86.333333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48211,2020-12-31 14:00:00,42.3,319.0,865.0,2010.0,0.75,2.94,11.7,34.0,27.5,...,2.066667,12.600000,40.666667,0.0,101170.000000,43.333333,1.700000,10.533333,76.333333,-0.033333
48212,2020-12-31 16:00:00,45.7,311.0,1080.0,2000.0,0.75,2.94,11.6,33.7,43.1,...,2.600000,12.400000,48.000000,0.0,101130.000000,30.000000,1.300000,10.200000,79.000000,-0.100000
48213,2020-12-31 18:00:00,48.6,301.0,1020.0,1950.0,0.75,2.96,11.6,31.7,34.8,...,2.400000,9.800000,55.333333,0.0,101143.333333,36.666667,2.366667,8.466667,84.333333,-0.033333
48214,2020-12-31 20:00:00,48.9,292.0,967.0,1910.0,0.75,2.98,11.5,33.1,37.2,...,2.433333,7.566667,63.666667,0.0,101130.000000,43.333333,2.933333,7.666667,86.666667,0.200000


In [103]:
mesures_test_Y = pd.DataFrame()
for curr_mois in range(1,13):
    mesures_mois = mesures_test[mesures_test["Date"].apply(lambda x: x.month == curr_mois)]
    mesures_mois = mesures_mois[liste_stations].groupby(mesures_mois["Date"].apply(lambda x: x.day)).mean()
    mesures_mois["Date"] = list(mesures_mois.index)
    mesures_mois["Date"] = pd.to_datetime("2020" \
                                          + "/" + str(curr_mois).rjust(2, "0") \
                                        + "/" + mesures_mois["Date"].apply(lambda x: str(x).rjust(2, "0")))
    mesures_test_Y = pd.concat([mesures_test_Y, mesures_mois], axis = 0)
    
mesures_test_Y = mesures_test_Y[(mesures_test_Y["Date"].apply(lambda x: x.month != 1)) \
                                | (mesures_test_Y["Date"].apply(lambda x: x.day >= 15))]

In [104]:
mesures_test_Y.to_csv("../Data/Base/Mesures_Test_Y.csv",
                     index = False)
mesures_test_Y

,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,K0910050,...,H0400010,H0400020,H0800011,H0800012,H0810010,H1700010,H3930020,H4340020,H8100021,Date
Date,,,,,,,,,,,,,,,,,,,,,
15,32.558333,125.416667,303.500000,518.833333,0.962500,3.465000,9.922500,30.966667,33.968333,58.808333,...,27.366667,15.458333,12.450000,11.283333,28.841667,105.083333,290.583333,298.333333,648.750000,2020-01-15
16,34.158333,116.583333,322.416667,507.250000,0.943333,3.218333,9.325000,29.525000,34.380000,40.100000,...,25.975000,14.091667,11.608333,11.016667,27.100000,97.091667,267.083333,274.583333,611.416667,2020-01-16
17,36.283333,128.500000,319.000000,546.166667,0.958333,3.280000,9.489167,27.950000,35.800000,53.966667,...,24.900000,13.033333,10.725000,10.733333,25.416667,90.841667,261.000000,270.416667,573.083333,2020-01-17
18,38.525000,138.750000,345.583333,534.583333,0.927500,3.451667,11.091667,28.375000,70.450000,44.008333,...,23.483333,11.958333,10.051667,10.508333,24.000000,86.033333,249.666667,255.666667,561.333333,2020-01-18
19,34.666667,138.333333,335.083333,575.500000,0.888333,3.070000,9.660000,26.816667,68.816667,65.058333,...,22.225000,10.983333,8.877500,10.608333,22.566667,80.850000,238.916667,250.416667,521.833333,2020-01-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,41.291667,188.250000,378.833333,627.666667,0.672500,3.170000,13.083333,30.383333,69.258333,24.091667,...,41.558333,15.008333,8.342500,15.566667,21.225000,47.258333,226.666667,256.500000,595.083333,2020-12-27
28,51.316667,243.500000,425.666667,676.250000,0.683333,3.257500,13.516667,32.983333,85.591667,76.791667,...,45.491667,15.666667,9.336667,17.275000,22.583333,48.200000,248.000000,302.250000,711.666667,2020-12-28
29,47.250000,552.666667,1474.750000,1975.833333,0.702500,3.126667,13.058333,35.491667,83.050000,108.750000,...,50.633333,24.841667,12.816667,20.991667,26.108333,50.350000,308.833333,353.666667,846.833333,2020-12-29


## Ensemble d'entrainement

In [105]:
mesures_train = mesures[mesures["Date"].apply(lambda x: x.year < 2020)]
mesures_train

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Vitesse_vent_07747,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Direction_vent_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01 00:00:00,41.3,122.0,285.0,506.0,0.42,2.87,9.33,22.8,6.90,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
1,2010-01-01 02:00:00,47.4,125.0,291.0,520.0,0.44,2.86,9.93,21.6,7.01,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
2,2010-01-01 04:00:00,49.2,132.0,300.0,516.0,0.50,2.84,10.80,21.9,7.03,...,11.800000,10.000000,61.000000,0.000000,99180.000000,220.000000,9.800000,14.300000,90.000000,0.000000
3,2010-01-01 06:00:00,42.4,135.0,314.0,496.0,0.54,2.90,10.20,20.8,7.04,...,9.066667,10.533333,50.333333,0.000000,99193.333333,240.000000,6.666667,12.900000,78.000000,0.533333
4,2010-01-01 08:00:00,47.3,132.0,329.0,472.0,0.54,3.04,10.70,21.4,7.12,...,9.066667,11.200000,43.000000,0.000000,99253.333333,253.333333,9.066667,11.800000,74.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,2019-12-31 14:00:00,57.8,205.0,621.0,1110.0,1.53,6.48,19.90,46.2,89.60,...,1.533333,13.133333,68.666667,0.000000,103073.333333,243.333333,1.800000,12.366667,70.333333,0.000000
43820,2019-12-31 16:00:00,64.5,206.0,634.0,1110.0,1.53,6.46,19.90,43.7,89.30,...,2.400000,12.400000,74.000000,0.000000,103040.000000,310.000000,1.200000,12.500000,67.000000,0.000000
43821,2019-12-31 18:00:00,67.2,201.0,635.0,1100.0,1.52,6.45,19.70,42.8,89.00,...,1.866667,8.800000,84.666667,0.000000,103066.666667,136.666667,1.733333,10.833333,74.333333,0.000000
43822,2019-12-31 20:00:00,56.3,198.0,625.0,1100.0,1.50,6.42,19.50,45.2,88.70,...,1.633333,6.666667,89.666667,0.000000,103073.333333,50.000000,2.400000,9.100000,81.000000,0.000000


In [106]:
mesures_train_X = mesures_train.copy()
mesures_train_X.to_csv("../Data/Base/Mesures_Train_X.csv",
                     index = False)
mesures_train_X

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Vitesse_vent_07747,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Direction_vent_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01 00:00:00,41.3,122.0,285.0,506.0,0.42,2.87,9.33,22.8,6.90,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
1,2010-01-01 02:00:00,47.4,125.0,291.0,520.0,0.44,2.86,9.93,21.6,7.01,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
2,2010-01-01 04:00:00,49.2,132.0,300.0,516.0,0.50,2.84,10.80,21.9,7.03,...,11.800000,10.000000,61.000000,0.000000,99180.000000,220.000000,9.800000,14.300000,90.000000,0.000000
3,2010-01-01 06:00:00,42.4,135.0,314.0,496.0,0.54,2.90,10.20,20.8,7.04,...,9.066667,10.533333,50.333333,0.000000,99193.333333,240.000000,6.666667,12.900000,78.000000,0.533333
4,2010-01-01 08:00:00,47.3,132.0,329.0,472.0,0.54,3.04,10.70,21.4,7.12,...,9.066667,11.200000,43.000000,0.000000,99253.333333,253.333333,9.066667,11.800000,74.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,2019-12-31 14:00:00,57.8,205.0,621.0,1110.0,1.53,6.48,19.90,46.2,89.60,...,1.533333,13.133333,68.666667,0.000000,103073.333333,243.333333,1.800000,12.366667,70.333333,0.000000
43820,2019-12-31 16:00:00,64.5,206.0,634.0,1110.0,1.53,6.46,19.90,43.7,89.30,...,2.400000,12.400000,74.000000,0.000000,103040.000000,310.000000,1.200000,12.500000,67.000000,0.000000
43821,2019-12-31 18:00:00,67.2,201.0,635.0,1100.0,1.52,6.45,19.70,42.8,89.00,...,1.866667,8.800000,84.666667,0.000000,103066.666667,136.666667,1.733333,10.833333,74.333333,0.000000
43822,2019-12-31 20:00:00,56.3,198.0,625.0,1100.0,1.50,6.42,19.50,45.2,88.70,...,1.633333,6.666667,89.666667,0.000000,103073.333333,50.000000,2.400000,9.100000,81.000000,0.000000


In [111]:
mesures_train_Y = pd.DataFrame()
for curr_annee in range(2010, 2020):
    mesures_annee = mesures_train[mesures_train["Date"].apply(lambda x: x.year == curr_annee)]
    for curr_mois in range(1,13):
        mesures_mois = mesures_annee[mesures_annee["Date"].apply(lambda x: x.month == curr_mois)]
        mesures_mois = mesures_mois[liste_stations].groupby(mesures_mois["Date"].apply(lambda x: x.day)).mean()
        mesures_mois["Date"] = list(mesures_mois.index)
        mesures_mois["Date"] = pd.to_datetime(str(curr_annee) \
                                              + "/" + str(curr_mois).rjust(2, "0") \
                                              + "/" + mesures_mois["Date"].apply(lambda x: str(x).rjust(2, "0")))
        mesures_train_Y = pd.concat([mesures_train_Y, mesures_mois], axis = 0)

In [113]:
mesures_train_Y.to_csv("../Data/Base/Mesures_Train_Y.csv",
                     index = False)

mesures_train_Y

,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,K0910050,...,H0400010,H0400020,H0800011,H0800012,H0810010,H1700010,H3930020,H4340020,H8100021,Date
Date,,,,,,,,,,,,,,,,,,,,,
1,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,13.575000,...,60.041667,9.601667,24.050000,13.166667,30.041667,59.700000,317.333333,302.666667,730.166667,2010-01-01
2,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,13.691667,...,57.666667,9.186667,22.250000,12.458333,31.425000,66.425000,345.500000,331.250000,736.833333,2010-01-02
3,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,13.708333,...,53.583333,9.015833,21.041667,12.025000,30.691667,68.141667,306.916667,314.666667,783.083333,2010-01-03
4,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,13.750000,...,47.783333,9.050833,20.616667,11.833333,30.175000,65.316667,269.916667,261.000000,687.333333,2010-01-04
5,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,19.825000,...,43.241667,9.143333,19.225000,13.016667,29.958333,62.350000,245.750000,239.750000,691.083333,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,142.416667,...,93.933333,57.991667,64.483333,30.633333,81.858333,158.416667,545.083333,571.250000,1150.833333,2019-12-27
28,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,159.500000,...,106.333333,60.125000,70.283333,32.550000,86.058333,164.583333,560.916667,577.666667,1180.000000,2019-12-28
29,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,151.750000,...,113.000000,67.033333,66.983333,31.325000,92.716667,167.916667,588.916667,607.666667,1224.166667,2019-12-29


## Ensembles de validation croisée

In [114]:
mesures_train_X = pd.read_csv("../Data/Base/Mesures_Train_X.csv")
mesures_train_X["Date"] = pd.to_datetime(mesures_train_X["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_train_X

,Date,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,...,Vitesse_vent_07747,Temperature_07747,Humidite_07747,Precipitations_07747,Pression_07761,Direction_vent_07761,Vitesse_vent_07761,Temperature_07761,Humidite_07761,Precipitations_07761
0,2010-01-01 00:00:00,41.3,122.0,285.0,506.0,0.42,2.87,9.33,22.8,6.90,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
1,2010-01-01 02:00:00,47.4,125.0,291.0,520.0,0.44,2.86,9.93,21.6,7.01,...,7.733333,10.333333,71.000000,0.133333,99393.333333,206.666667,8.400000,14.233333,90.000000,0.000000
2,2010-01-01 04:00:00,49.2,132.0,300.0,516.0,0.50,2.84,10.80,21.9,7.03,...,11.800000,10.000000,61.000000,0.000000,99180.000000,220.000000,9.800000,14.300000,90.000000,0.000000
3,2010-01-01 06:00:00,42.4,135.0,314.0,496.0,0.54,2.90,10.20,20.8,7.04,...,9.066667,10.533333,50.333333,0.000000,99193.333333,240.000000,6.666667,12.900000,78.000000,0.533333
4,2010-01-01 08:00:00,47.3,132.0,329.0,472.0,0.54,3.04,10.70,21.4,7.12,...,9.066667,11.200000,43.000000,0.000000,99253.333333,253.333333,9.066667,11.800000,74.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,2019-12-31 14:00:00,57.8,205.0,621.0,1110.0,1.53,6.48,19.90,46.2,89.60,...,1.533333,13.133333,68.666667,0.000000,103073.333333,243.333333,1.800000,12.366667,70.333333,0.000000
43820,2019-12-31 16:00:00,64.5,206.0,634.0,1110.0,1.53,6.46,19.90,43.7,89.30,...,2.400000,12.400000,74.000000,0.000000,103040.000000,310.000000,1.200000,12.500000,67.000000,0.000000
43821,2019-12-31 18:00:00,67.2,201.0,635.0,1100.0,1.52,6.45,19.70,42.8,89.00,...,1.866667,8.800000,84.666667,0.000000,103066.666667,136.666667,1.733333,10.833333,74.333333,0.000000
43822,2019-12-31 20:00:00,56.3,198.0,625.0,1100.0,1.50,6.42,19.50,45.2,88.70,...,1.633333,6.666667,89.666667,0.000000,103073.333333,50.000000,2.400000,9.100000,81.000000,0.000000


In [128]:
train_split_X = pd.DataFrame()
for i in range(9):
    train_split_X["Train_" + str(i)] = mesures_train_X["Date"].apply(lambda x: x.year <= 2010+i)
for i in range(9):
    train_split_X["Test_" + str(i)] = mesures_train_X["Date"].apply(lambda x: x.year == 2010+i+1)
train_split_X

,Train_0,Train_1,Train_2,Train_3,Train_4,Train_5,Train_6,Train_7,Train_8,Test_0,Test_1,Test_2,Test_3,Test_4,Test_5,Test_6,Test_7,Test_8
0,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
1,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
2,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
3,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
4,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
43820,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
43821,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
43822,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [129]:
train_split_X.to_csv("../Data/Base/Index_CV_X.csv",
                    index=False)

In [130]:
mesures_train_Y = pd.read_csv("../Data/Base/Mesures_Train_Y.csv")
mesures_train_Y["Date"] = pd.to_datetime(mesures_train_Y["Date"], format = "%Y/%m/%d %H:%M:%S")
mesures_train_Y

,O0200020,O2620010,O6140010,O9000010,K0030020,K0100020,K0260020,K0550010,K0690010,K0910050,...,H0400010,H0400020,H0800011,H0800012,H0810010,H1700010,H3930020,H4340020,H8100021,Date
0,42.375000,128.000000,350.500000,490.583333,0.535000,3.119167,11.005000,24.850000,8.470833,13.575000,...,60.041667,9.601667,24.050000,13.166667,30.041667,59.700000,317.333333,302.666667,730.166667,2010-01-01
1,38.458333,106.175000,447.666667,666.333333,0.467500,2.952500,10.812500,29.208333,8.045000,13.691667,...,57.666667,9.186667,22.250000,12.458333,31.425000,66.425000,345.500000,331.250000,736.833333,2010-01-02
2,35.891667,104.941667,385.000000,690.333333,0.423333,2.385833,8.619167,21.000000,7.295833,13.708333,...,53.583333,9.015833,21.041667,12.025000,30.691667,68.141667,306.916667,314.666667,783.083333,2010-01-03
3,34.041667,92.708333,343.333333,607.833333,0.424167,2.447500,8.489167,21.616667,42.940000,13.750000,...,47.783333,9.050833,20.616667,11.833333,30.175000,65.316667,269.916667,261.000000,687.333333,2010-01-04
4,37.175000,109.258333,357.083333,616.666667,0.421667,2.426667,8.027500,23.250000,56.650000,19.825000,...,43.241667,9.143333,19.225000,13.016667,29.958333,62.350000,245.750000,239.750000,691.083333,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,81.958333,292.583333,955.000000,1720.833333,1.997500,9.271667,28.816667,60.675000,102.166667,142.416667,...,93.933333,57.991667,64.483333,30.633333,81.858333,158.416667,545.083333,571.250000,1150.833333,2019-12-27
3648,75.508333,258.250000,849.583333,1530.833333,1.912500,8.575000,27.591667,61.533333,102.983333,159.500000,...,106.333333,60.125000,70.283333,32.550000,86.058333,164.583333,560.916667,577.666667,1180.000000,2019-12-28
3649,69.008333,236.333333,749.416667,1375.000000,1.784167,7.627500,24.133333,53.983333,93.641667,151.750000,...,113.000000,67.033333,66.983333,31.325000,92.716667,167.916667,588.916667,607.666667,1224.166667,2019-12-29
3650,63.858333,215.000000,682.000000,1245.000000,1.650833,6.958333,21.900000,49.550000,90.725000,127.916667,...,114.083333,68.925000,70.533333,32.716667,94.666667,173.666667,599.000000,602.416667,1228.333333,2019-12-30


In [133]:
train_split_Y = pd.DataFrame()
for i in range(9):
    train_split_Y["Train_" + str(i)] = mesures_train_Y["Date"].apply(lambda x: x.year <= 2010+i)
    train_split_Y["Train_" + str(i)].iloc[mesures_train_Y["Date"].apply(lambda x: x.month == 1) & mesures_train_Y["Date"].apply(lambda x: x.day < 15)] = False # On enlève les 14 premiers jours
    
for i in range(9):
    train_split_Y["Test_" + str(i)] = mesures_train_Y["Date"].apply(lambda x: x.year == 2010+i+1)
    train_split_Y["Test_" + str(i)].iloc[mesures_train_Y["Date"].apply(lambda x: x.month == 1) & mesures_train_Y["Date"].apply(lambda x: x.day < 15)] = False  # On enlève les 14 premiers jours
train_split_Y

,Train_0,Train_1,Train_2,Train_3,Train_4,Train_5,Train_6,Train_7,Train_8,Test_0,Test_1,Test_2,Test_3,Test_4,Test_5,Test_6,Test_7,Test_8
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3648,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3649,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3650,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [134]:
train_split_Y.to_csv("../Data/Base/Index_CV_Y.csv",
                    index=False)